In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# EXERCISE 1 SOLUTION
# Define a kernel that prepares the walker qubits in an
# equal superposition of the states |2> = |0010> and |3> = |0011>.

@cudaq.kernel
def initial_position(qubits : cudaq.qvector):
    """ Apply gates to the qubits to prepare the GHZ state
    Parameters
        qubits: cudaq.qvector
        qubits for the walker
    """
    # Edit the code below this line
    x(qubits[2])
    h(qubits[3])
    
    # Edit the code above this line

In [ ]:
# EXERCISE 2 SOLUTION

# Define a kernel on 4 qubits for the DEC operation that
# maps |x> to |x-1> mod 16 and verify that it works as expected for |0001>

# Define the decrementer matrix

def decrementer(num_qubits):
    size = 2**num_qubits
    dec_matrix = np.zeros((size, size))
    for i in range(size):
        dec_matrix[i, (i + 1) % size] = 1
    return dec_matrix

# EDIT THE CODE BELOW THIS LINE

# Create a custom register operation for the decrementer called DEC
cudaq.register_operation("DEC", decrementer(num_qubits))

# EDIT THE CODE ABOVE THIS LINE

# Create a kernel that applies the DEC to the 4-qubit state |0001>
@cudaq.kernel
def check_decrementer_kernel():
    qubits = cudaq.qvector(4)
    # Initialize the qubits to |0001>
    x(qubits[3])
    # Apply the decrementer operation 
    DEC(qubits[0], qubits[1], qubits[2], qubits[3])

result = cudaq.sample(check_decrementer_kernel, shots_count=1000).most_probable()
print('Decrementer kernel |0001> -> |{}>'.format(result))

In [ ]:
# EXERCISE 3 SOLUTION
# Complete the code below to define S-
# S- will apply the DEC operation to the walker qubits
# when the coin qubit is in the |0> state

# Set the number of qubits
num_qubits = 4

@cudaq.kernel
def DTQW_one_step(num_qubits: int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qubit()
    endpoint_qubit = cudaq.qubit()

    # Initial walker state 1/(sqrt{2}) ( |2>+|3>)
    initial_position(walker_qubits)
        
    # Initial coin state    
    h(coin_qubit) #Comment to set the initial coin state to be |0>
        
    # One quantum walk step
    # Coin operation F=H
    h(coin_qubit)
    
    # Walker's position change 
    
    ## Shifting right
    
    # Avoid shifting from |1111> to |0000> in case the coin is |1> by flipping the coin to |0>
    no_INC_at_right_endpoint(walker_qubits, coin_qubit, endpoint_qubit)
    
    # Shift right (S+) when the coin is |1> and the walker is not in the |1111> state
    INC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])

    # Reset the coin and endpoints in case they were changed to avoid moving from |1111> to |0000>
    reset_coin_and_endpoint(coin_qubit, endpoint_qubit)
    
    ## Shifting left
    
    # Avoid shifting from |0000> to |1111> in case the coin is |0> by flipping the coin to |1>
    no_DEC_at_left_endpoint(walker_qubits, coin_qubit, endpoint_qubit)
    
    # Shift left (S-) when the coin is |0>
    # EDIT CODE BELOW THIS LINE
    
    x(coin_qubit)
    DEC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])
    x(coin_qubit)
    
    # EDIT CODE ABOVE THIS LINE
    
    # Reset the coin and endpoints in case they were changed to avoid moving from |0000> to |1111>
    reset_coin_and_endpoint(coin_qubit, endpoint_qubit)
    
    mz(walker_qubits)

# Visualize the kernel for the quantum walk
#print(cudaq.draw(DTQW_one_step, num_qubits))

# Sample the kernel for the quantum walk
result = cudaq.sample(DTQW_one_step, num_qubits, shots_count=1000)
print(result)
plot_results(result, num_qubits)